Columns are Default: element, type, text, name
Filtered by: Isolated Areas , Leistung != 0

In [30]:
import pandas as pd
import re

# Constants
DEFAULT_VALUES = {
    'element': 'area',
    'type': 'warning',
    'text': 'Isoliertes Gebiet',
    'name': 'isolated area'
}

def load_data(file_path, sheet_name='Sheet1'):
    return pd.read_excel(file_path, sheet_name=sheet_name)

def filter_isolated_areas(df):
    return df.loc[df['name'] == DEFAULT_VALUES['name']]

def extract_leistung(load_column):
    return load_column.str.extract(r'p_mw\': (\d+\.\d+)', expand=False).astype(float).round(4)

def fill_template(df, data, default_values):
    df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
    df = df.fillna(default_values)
    return df

def reorder_columns(df):
    return df[['element_index'] + ['element', 'level'] + [col for col in df.columns if col not in ['element', 'element_index', 'level']]]

# Load data from Excel file
net_areas_file = load_data('net_areas.xlsx')

# Filter isolated areas without zero load
isolated_areas = filter_isolated_areas(net_areas_file)
isolated_areas_without_zero_load = isolated_areas[isolated_areas['load'].notnull()].copy()  # Explicitly create a copy

# Extract 'Leistung' values using vectorized regex
isolated_areas_without_zero_load['Leistung'] = extract_leistung(isolated_areas_without_zero_load['load'])

# Prepare data to add to the template DataFrame
data_to_add = {
    'element': DEFAULT_VALUES['element'],
    'type': DEFAULT_VALUES['type'],
    'text': DEFAULT_VALUES['text'],
    'name': DEFAULT_VALUES['name'],
    'element_index': isolated_areas_without_zero_load.index,
    'Leistung': isolated_areas_without_zero_load['Leistung'].tolist(),
    'level': isolated_areas_without_zero_load['level'].tolist(),  # Include 'level' values
}

# Fill the template DataFrame
template_df = fill_template(pd.DataFrame(), data_to_add, DEFAULT_VALUES)

# Filter 'Leistung' column to remove rows with NaN values and where 'Leistung' is 0
template_df = template_df[(template_df['Leistung'].notna()) & (template_df['Leistung'] != 0)]

# Reset the index
template_df = template_df.reset_index(drop=True)

# Reorder columns with 'element_index' as the second column
template_df = reorder_columns(template_df)

# Display the updated DataFrame
print(template_df.head())


   element_index element level     type               text           name  \
0              0    area    lv  warning  Isoliertes Gebiet  isolated area   
1              1    area    lv  warning  Isoliertes Gebiet  isolated area   
2              2    area    lv  warning  Isoliertes Gebiet  isolated area   
3              7    area    lv  warning  Isoliertes Gebiet  isolated area   
4             14    area    lv  warning  Isoliertes Gebiet  isolated area   

   Leistung  
0    0.0536  
1    0.0356  
2    0.0128  
3    0.0328  
4    0.1965  


In [31]:
template_df.describe()

,element_index,Leistung
count,483.000000,483.000000
mean,4723.151139,0.044849
std,1058.891021,0.102392
min,0.000000,0.000200
25%,4547.500000,0.005400
50%,4726.000000,0.013900
75%,5402.500000,0.038300
max,5937.000000,1.404000


In [32]:
# Filter rows for 'lv' level
lv_rows = template_df[template_df['level'] == 'lv']
print("Rows with 'lv' level:")
print(len(lv_rows))

# Filter rows for 'mv' level
mv_rows = template_df[template_df['level'] == 'mv']
print("\nRows with 'mv' level:")
print(len(mv_rows))

# Filter rows for 'hv' level
hv_rows = template_df[template_df['level'] == 'hv']
print("\nRows with 'hv' level:")
print(len(hv_rows))


Rows with 'lv' level:
483

Rows with 'mv' level:
0

Rows with 'hv' level:
0


In [33]:
template_df.to_csv('Neo_Data_Check.csv')